In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col

In [20]:
# Create a Spark session
spark = SparkSession.builder.appName("SQL tasks").getOrCreate()
# load data

Task 2.1

In [10]:
CustomerDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Customer.csv")
PurchaseDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Purchase.csv")
PurchaseDF.printSchema()
CustomerDF.printSchema()
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal>=600""")
T1.printSchema()
T1.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T1.csv")
T1.show(10)

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- Salary: double (nullable = true)

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

+-------+------+------------------+-------------+--------------------+
|TransID|CustID|        TransTotal|TransNumItems|           TransDesc|
+-------+------+------------------+-------------+--------------------+
|      1| 38695|1120.3707736244614|            7|Pet: Food Toy Bed...|
|      2| 37410|1258.8819884338257|           10|Clothes: Watch Sc...|
|      3| 16007| 969.2

Task 2.2

In [13]:
CustomerDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Customer.csv")
PurchaseDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Purchase.csv")
PurchaseDF.printSchema()
CustomerDF.printSchema()
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal>=600""")
T1.createOrReplaceTempView("T1View")
T2=spark.sql("""SELECT TransNumItems,round(PERCENTILE_APPROX(TransTotal,0.5),2) AS MEDIANTOTAL, round(MIN(TransTotal),2) AS MINTOTAL, round(MAX(TransTotal),2) AS MAXTOTAL FROM T1View GROUP BY TransNumItems""")
T2.show(10)
T2.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T2.csv")

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- Salary: double (nullable = true)

+-------------+-----------+--------+--------+
|TransNumItems|MEDIANTOTAL|MINTOTAL|MAXTOTAL|
+-------------+-----------+--------+--------+
|           12|     1298.5|  600.01| 1999.99|
|            1|    1298.26|   600.0| 1999.99|
|           13|    1298.73|   600.0|  2000.0|
|            6|    1300.19|   600.0| 1999.99|
|            3|    1300.14|  600.02|  2000.0|
|            5|    1301.46|  600.01| 1999.99|
|           15|    1299.34|   600.0|  2000.0|
|            9|    1301.55|  600.01|  2000.0|
|            4|    1301.51|  600.01|  2000.0|
|            8|    1299.35|   600.0|  2

Task 2.3

In [17]:
CustomerDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Customer.csv")
PurchaseDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Purchase.csv")
PurchaseDF.printSchema()
CustomerDF.printSchema()
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal>=600""")
T1.createOrReplaceTempView("T1View")
CustomerDF.where("Age between 18 and 25").createOrReplaceTempView("YoungCustomerView")
T3=spark.sql("""SELECT cus.ID, cus.AGE, round(SUM(TransTotal),2) AS TotalAmount,SUM(TransNumItems) AS TotalNumberOfItems FROM T1View t1 INNER JOIN YoungCustomerView cus ON t1.CustID=cus.ID GROUP BY cus.ID, cus.AGE""")
T3.show(10)
T3.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T3.csv")

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- Salary: double (nullable = true)

+-----+---+-----------+------------------+
|   ID|AGE|TotalAmount|TotalNumberOfItems|
+-----+---+-----------+------------------+
|27877| 19|   98547.53|               570|
|31183| 19|  113362.21|               722|
| 5031| 21|   92596.82|               565|
|18052| 24|  113270.29|               631|
| 9976| 22|  101693.71|               616|
|39653| 23|    87632.2|               488|
|23849| 18|   85150.51|               513|
|36372| 24|   114000.2|               727|
|41048| 18|   84043.42|               529|
|27451| 25|   80973.72|               515|
+-----+---+-----------+---------

Task 2.4

In [21]:
CustomerDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Customer.csv")
PurchaseDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Purchase.csv")
PurchaseDF.printSchema()
CustomerDF.printSchema()
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal>=600""")
T1.createOrReplaceTempView("T1View")
#CustomerDF.where("Age between 18 and 25").createOrReplaceTempView("YoungCustomerView")
CustomerDF.createOrReplaceTempView("CustomerView")
YoungCustomer=spark.sql("""SELECT * FROM CustomerView C WHERE Age between 18 and 25""")
YoungCustomer.createOrReplaceTempView("YoungCustomerView")
T3=spark.sql("""SELECT cus.ID, cus.AGE, round(SUM(TransTotal),2) AS TotalAmount,SUM(TransNumItems) AS TotalNumberOfItems FROM T1View t1 INNER JOIN YoungCustomerView cus ON t1.CustID=cus.ID GROUP BY cus.ID, cus.AGE""")
T3.createOrReplaceTempView("T3View")
T4=spark.sql("""SELECT c1.ID as C1_ID, c2.ID as C2_ID, c1.Age as Age1, c2.age as Age2, round(c1.TotalAmount,2) as TotalAmount1, round(c2.TotalAmount,2) TotalAmount2, c1.TotalNumberOfItems  as TotalNumberOfItems1, c2.TotalNumberOfItems as TotalNumberOfItems2 FROM T3View c1 inner join T3View c2 on c1.Age<c2.Age where c1.TotalAmount>c2.TotalAmount and c1.TotalNumberOfItems<c2.TotalNumberOfItems""")
T4.show(10)
T4.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T4.csv")


root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- Salary: double (nullable = true)

+-----+-----+----+----+------------+------------+-------------------+-------------------+
|C1_ID|C2_ID|Age1|Age2|TotalAmount1|TotalAmount2|TotalNumberOfItems1|TotalNumberOfItems2|
+-----+-----+----+----+------------+------------+-------------------+-------------------+
|27877|34579|  19|  20|    98547.53|    98438.62|                570|                634|
|27877|47510|  19|  21|    98547.53|     96325.5|                570|                658|
|27877|11156|  19|  23|    98547.53|    97483.07|                570|                636|
|27877| 8797|  19|  21|    98547.53|    93278.21|  

In [18]:
spark.sparkContext.stop()